>For Googla Colab Only

>>git clone https://github.com/OperationalizingAI/Hackathon-2-22-24.gi

In [ ]:
%pip install --quiet langchain pypdf pymongo langchain-openai tiktoken google-cloud-secret-manager
%pip install --quiet pandas sentence-transformers scikit-learn numpy

In [ ]:
%pip install --upgrade google-auth

In [ ]:
import os

%pip show langchain

from platform import python_version
print(python_version())

Name: langchain
Version: 0.1.6
Summary: Building applications with LLMs through composability
Home-page: https://github.com/langchain-ai/langchain
Author: 
Author-email: 
License: MIT
Location: /usr/local/lib/python3.10/dist-packages
Requires: aiohttp, async-timeout, dataclasses-json, jsonpatch, langchain-community, langchain-core, langsmith, numpy, pydantic, PyYAML, requests, SQLAlchemy, tenacity
Required-by: 
3.10.12


In [ ]:
from google.cloud import secretmanager
from google.colab import auth
from google.colab import drive

In [ ]:
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
def load_secrets(secrets_name, project_id):
  # Build a client
  auth.authenticate_user()
  client = secretmanager.SecretManagerServiceClient()
  secret_name = secrets_name
  # Create path to latest secret
  resource_name = f"projects/{project_id}/secrets/{secret_name}/versions/latest"
  # Get your secret :
  response = client.access_secret_version(request={"name": resource_name})
  secret_string = response.payload.data.decode('UTF-8')
  return secret_string

In [ ]:
project_id = 'botchagalupep1'
openai_api_key = load_secrets("openai_api_key",project_id)
os.environ['OPENAI_API_KEY'] = openai_api_key
#MONGODB_ATLAS_CLUSTER_URI = load_secrets("mdb_uri",project_id)
MONGODB_ATLAS_CLUSTER_URI = load_secrets("MDB_CLUSTER0_URI",project_id)
langsmith_api_key = load_secrets("langsmith_api_key",project_id)
#print(langsmith_api_key )

Now we will setup the environment variables for the MongoDB Atlas cluster


 In the below example, embedding is the name of the field that contains the embedding vector. Please refer to the documentation to get more details on how to define an Atlas Vector Search index. You can name the index {ATLAS_VECTOR_SEARCH_INDEX_NAME} and create the index on the namespace {DB_NAME}.{COLLECTION_NAME}. Finally, write the following definition in the JSON editor on MongoDB Atlas:

~~~
{
 "fields": [{
   "type": "vector",
   "path": "embedding",
   "numDimensions": 1536,
   "similarity": "cosine"
 }]
}



#Load Data

In [ ]:
from langchain_community.document_loaders import PyPDFLoader

# Load the PDF
loader = PyPDFLoader("https://arxiv.org/pdf/2303.08774.pdf")
#loader = PyPDFLoader("/content/gdrive/MyDrive/GAI/transcripts/DEM.pdf")
#loader = PyPDFLoader("/content/gdrive/MyDrive/GAI/transcripts/dev-final-entities-db.pdf")
#loader = PyPDFLoader("/content/gdrive/MyDrive/GAI/transcripts/scatterly-parsed-json.pdf")
#loader = PyPDFLoader("/content/gdrive/MyDrive/GAI/transcripts/opencontext-retail.pdf")
#loader = PyPDFLoader("/content/opencontext.pdf")
data = loader.load()

In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity

import pandas as pd
import numpy as np

# Initialize the sentence transformer model for embeddings
model = SentenceTransformer('all-MiniLM-L6-v2')

def tokenize_text(method, text, chunk_size, chunk_overlap, num_chunks):
    """
    Tokenizes the input text based on the selected method and provided parameters.
    """
    num_chunks = int(num_chunks)
    output = []

    # Ensure text is provided
    if not text.strip():
        return pd.DataFrame(columns=['Chunk #', 'Text Chunk', 'Character Count', 'Token Count'])

    if method == "RecursiveCharacterTextSplitter":
        text_splitter = RecursiveCharacterTextSplitter(chunk_size=chunk_size, chunk_overlap=chunk_overlap, length_function=len, is_separator_regex=False)
        tokenized_texts = text_splitter.split_text(text)[:num_chunks]
        for i, chunk in enumerate(tokenized_texts):
            output.append({
                'Chunk #': i,
                'Text Chunk': chunk,
                'Character Count': len(chunk),
                'Token Count': len(chunk.split())
            })

    df = pd.DataFrame(output)
    return df

def calculate_embeddings(df):
    """
    Calculates embeddings for each text chunk in the dataframe.
    """
    if df.empty:
        return df

    chunks = df['Text Chunk'].tolist()
    embeddings = model.encode(chunks)
    df['Embeddings'] = embeddings.tolist()
    return df

def search_similar_chunks(query, df_with_embeddings):
    """
    Search for chunks similar to the query embedding.
    """
    # Compute the query embedding
    query_embedding = model.encode([query])[0]

    # Calculate similarity scores
    chunk_embeddings = np.vstack(df_with_embeddings['Embeddings'])
    similarity_scores = cosine_similarity([query_embedding], chunk_embeddings)[0]

    # Insert similarity scores into the dataframe after 'Chunk #'
    df_with_embeddings.insert(1, 'Similarity', similarity_scores)

    # Return the dataframe sorted by similarity scores in descending order
    return df_with_embeddings.sort_values(by='Similarity', ascending=False)

def process_and_embed(method, text, chunk_size, chunk_overlap, num_chunks):
    """
    Tokenizes the text and calculates embeddings.
    """
    df = tokenize_text(method, text, chunk_size, chunk_overlap, num_chunks)
    df_with_embeddings = calculate_embeddings(df)
    return df_with_embeddings

def update_output(method, text, chunk_size, chunk_overlap, num_chunks, query):
    df_with_embeddings = process_and_embed(method, text, chunk_size, chunk_overlap, num_chunks)
    if query:
        df_with_embeddings = search_similar_chunks(query, df_with_embeddings)
        # Update the headers to reflect the new column order after similarity search
        return df_with_embeddings[['Chunk #', 'Similarity', 'Text Chunk', 'Character Count', 'Token Count', 'Embeddings']]
    return df_with_embeddings[['Chunk #', 'Text Chunk', 'Character Count', 'Token Count', 'Embeddings']]


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
from builtins import len

for i, chunk in enumerate(data):
    if i >= 5:  # Stop after processing 5 chunks
        break
    print("Chunk Number:", i, "\n")
    #print("Page Content:", chunk.page_content, "\n")
    print("Character Count:", len(str(chunk)), "\n")
    print("Token Count:", len(str(chunk).split()), "\n")
    #print("Metadata:", str(chunk.metadata), "\n")


Chunk Number: 0 

Character Count: 3564 

Token Count: 482 

Chunk Number: 1 

Character Count: 4130 

Token Count: 556 

Chunk Number: 2 

Character Count: 1222 

Token Count: 170 

Chunk Number: 3 

Character Count: 3455 

Token Count: 483 

Chunk Number: 4 

Character Count: 2711 

Token Count: 502 



In [ ]:
method = "RecursiveCharacterTextSplitter"
chunk_size = 200
chunk_overlap = 25
num_chunks = 20
text = str(data)
query = "What is GPT-4"

update_output(method, text, chunk_size, chunk_overlap, num_chunks, query)


,Chunk #,Similarity,Text Chunk,Character Count,Token Count,Embeddings
2,2,0.713234,including passing a simulated\nbar exam with a...,181,30,"[-0.0499851293861866, 0.0031717480160295963, -..."
5,5,0.688439,predict some aspects of GPT-4’s\nperformance b...,199,26,"[-0.07324909418821335, -0.06115596368908882, -..."
17,17,0.678716,of GPT-4 (based on small runs trained in simil...,194,29,"[0.01012280210852623, 0.03480536490678787, -0...."
9,9,0.656033,more complex and nuanced scenarios. To test it...,199,29,"[-0.014145134016871452, 0.030883973464369774, ..."
1,1,0.644171,produce text outputs. While less capable than\...,192,23,"[0.030117619782686234, 0.018961912021040916, -..."
16,16,0.623004,learning infrastructure and\noptimization meth...,197,27,"[-0.002811617683619261, 0.008920865133404732, ..."
10,10,0.573714,it performs quite well and often outscores the...,192,33,"[-0.02528281882405281, -0.000942700426094234, ..."
0,0,0.537164,[Document(page_content='GPT-4 Technical Report...,200,22,"[-0.06292103976011276, 0.05016793683171272, -0..."
14,14,0.492087,"On translated variants of MMLU, GPT-4\nsurpass...",188,25,"[-0.07436112314462662, -0.012194906361401081, ..."
18,18,0.479275,"similar limitations to earlier GPT models [ 1,...",196,30,"[-0.027139713987708092, -0.049942053854465485,..."


In [ ]:
from pymongo import MongoClient

# initialize MongoDB python client
client = MongoClient(MONGODB_ATLAS_CLUSTER_URI)

DB_NAME = "Cluster0"
COLLECTION_NAME = "OpenContext0"
ATLAS_VECTOR_SEARCH_INDEX_NAME = "vector_index"

MONGODB_COLLECTION = client[DB_NAME][COLLECTION_NAME]

In [ ]:
from langchain_community.vectorstores import MongoDBAtlasVectorSearch
from langchain_openai import OpenAIEmbeddings

# insert the documents in MongoDB Atlas with their embedding
vector_search = MongoDBAtlasVectorSearch.from_documents(
    documents=docs,
    embedding=OpenAIEmbeddings(disallowed_special=()),
    collection=MONGODB_COLLECTION,
    index_name=ATLAS_VECTOR_SEARCH_INDEX_NAME,
)

In [ ]:
# Perform a similarity search between the embedding of the query and the embeddings of the documents
query = "what are the alerts "
results = vector_search.similarity_search(query)

print(results[0].page_content)

IndexError: list index out of range